In [ ]:
from keras import models
from keras import layers
from keras import activations
from keras import initializers
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import callbacks
from keras.preprocessing import image

: 

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(
    32, (5, 5),
    input_shape = (64, 64, 3)
))
model.add(layers.MaxPooling2D(
    pool_size = (2, 2)
))
model.add(layers.Conv2D(
    16, (5, 5),
    input_shape = (30, 30, 3)
))
model.add(layers.MaxPooling2D(
    pool_size = (2, 2)
))

model.add(layers.Flatten())
model.add(layers.Dense(256,
    kernel_initializer = initializers.RandomNormal(stddev = 1),
    bias_initializer = initializers.Zeros()))
model.add(layers.Activation(activations.relu))

model.add(layers.Dense(64,
    kernel_regularizer = regularizers.L1(1e-4),))
model.add(layers.Activation(activations.relu))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(64))
model.add(layers.Activation(activations.relu))

model.add(layers.Dense(2))
model.add(layers.Activation(activations.softmax))

In [ ]:
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=[metrics.CategoricalAccuracy(), metrics.Precision()]
)

In [ ]:
dataGen = image.ImageDataGenerator(
    rescale = 1.0/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = False,

    validation_split = 0.2
)

X_train = dataGen.flow_from_directory(
    'PetImages',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'training'
)

X_tests = dataGen.flow_from_directory(
    'PetImages',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'validation'
)

In [ ]:
total_data = 19999
batchSize = 32

model.fit(X_train,
          validation_data = X_tests,
          steps_per_epoch = total_data / batchSize,
          epochs = 50,
          validation_steps = (total_data / batchSize) / 10,
          callbacks = [
            callbacks.ModelCheckpoint(
                filepath = 'ModelCheckpoint/model.{epoch:02d}-{val_loss:.2f}.h5'
            )
          ]
)

model.save('model')